In [2]:
import parse_inventory as pi
import parse_dealership
import requests
import re
from bs4 import BeautifulSoup
import cloudscraper
import pandas as pd
from datetime import datetime
import sqlite3

In [3]:
headers = {
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36',
}

URL = 'https://www.newtonautoandsales.com/cars-for-sale'
response = requests.get(URL, headers = headers)
soup = BeautifulSoup(response.text, "html.parser")



In [4]:
print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta charset="utf-8"/>
  <meta content="telephone=no" name="format-detection"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <link href="https://cdn09.carsforsale.com/" rel="dns-prefetch"/>
  <title>
   Cars For Sale in Newton, MA - Newton Automotive and Sales
  </title>
  <meta content="Find   Cars listings for sale starting at $15480 in Newton, MA. Shop Newton Automotive and Sales to find great deals on   Cars listings." name="description">
   <script type="application/ld+json">
    {
  "@context": "http://schema.org",
  "@type": "AutoDealer",
  "name": "Newton Automotive and Sales",
  "telephone": "(617) 964-6136",
  "url": "https://www.newtonautoandsales.com/",
  "image": "https://assets-cc.carsforsale.com/461411/avatar.jpg",
  "address": {
    "@type": "PostalAddress",
    "streetAddress": "249 Centre St",
    "postalCode": "02458",
    "addressRegion":

In [5]:
miles_and_price = pi.get_numeric_vehicle_data(soup, 'div', 'vehicle-snapshot__main-info font-primary')
price = [el for idx, el in enumerate(miles_and_price) if idx % 2 == 0]
miles = [el for idx, el in enumerate(miles_and_price) if idx % 2 != 0]

In [6]:
price

[36590,
 39480,
 48560,
 29350,
 36850,
 17980,
 41580,
 48760,
 47480,
 34980,
 35480,
 40980,
 29850,
 31980,
 39980]

In [7]:
len(price)

15

In [8]:
miles

[15679,
 29520,
 53912,
 53298,
 99106,
 54283,
 62800,
 85521,
 29709,
 99848,
 12426,
 18474,
 57247,
 36776,
 29058]

In [9]:
vehicle_info = pi.parseColumn(soup, 'div', 'vehicle-snapshot__info-text-container')

In [10]:
vehicle_info

['Engine  2.0L I4 Turbocharger',
 'Exterior Color  Moonlight Blue Metallic',
 'Transmission  Automatic 7-Speed',
 'Interior Color  Nougat Brown Interior',
 'Drivetrain  AWD',
 'Fuel Economy  24 city / 34 hwy',
 'Engine  I4 2.0L Turbocharger',
 'Exterior Color  Nano Gray Metallic',
 'Transmission  Automatic 6-Speed',
 'Interior Color  Rock Gray, Fine Nappa Leather',
 'Drivetrain  AWD',
 'Fuel Economy  23 city / 30 hwy',
 'Engine  V6 3.0L Supercharger',
 'Exterior Color  Black',
 'Transmission  Automatic 8-Speed',
 'Interior Color  Brown',
 'Drivetrain  AWD',
 'Fuel Economy  19 city / 25 hwy',
 'Engine  I4 2.0L Turbocharger',
 'Exterior Color  White',
 'Transmission  Automatic 7-Speed',
 'Interior Color  Beige',
 'Drivetrain  AWD',
 'Fuel Economy  24 city / 31 hwy',
 'Engine  V6 3.0L Turbocharger',
 'Exterior Color  Black',
 'Transmission  Automatic 8-Speed',
 'Interior Color  Black',
 'Drivetrain  AWD',
 'Fuel Economy  19 city / 24 hwy',
 'Engine  I4 2.0L Turbocharger',
 'Exterior Color

In [11]:
engine = vehicle_info[::6]
engine = [el.replace('Engine', '').strip() for el in engine]

In [12]:
engine

['2.0L I4 Turbocharger',
 'I4 2.0L Turbocharger',
 'V6 3.0L Supercharger',
 'I4 2.0L Turbocharger',
 'V6 3.0L Turbocharger',
 'I4 2.0L Turbocharger',
 'V6 3.0L Turbocharger',
 'V8 4.0L Twin Turbocharger',
 'I4 2.0L Turbocharger',
 'V6 3.0L Supercharger',
 'I4 2.0L Turbocharger',
 'I4 2.0L Turbocharger',
 'I6 3.0L Turbocharger',
 'V10 5.0L Natural Aspiration',
 'I4 2.0L Turbocharger']

In [13]:
transmission = vehicle_info[2::6]
transmission = [el.replace('Transmission', '').strip() for el in transmission]
transmission

['Automatic 7-Speed',
 'Automatic 6-Speed',
 'Automatic 8-Speed',
 'Automatic 7-Speed',
 'Automatic 8-Speed',
 'Automatic 8-Speed',
 'Automatic 8-Speed',
 'Automatic 7-Speed',
 'Automatic 7-Speed',
 'Automatic 8-Speed',
 'Automatic 8-Speed',
 'Automatic 8-Speed',
 'Automatic 8-Speed',
 'Automatic 7-Speed',
 'Automatic 8-Speed']

In [14]:
interior = vehicle_info[3::6]
interior = [el.replace('Interior Color', '').strip() for el in interior]
interior

['Nougat Brown Interior',
 'Rock Gray, Fine Nappa Leather',
 'Brown',
 'Beige',
 'Black',
 'Black',
 'Black',
 'Black',
 'Nougat Brown, Leather Seating',
 'Black',
 'Black',
 'Black, Dakota Leather',
 'Black, Dakota Leather',
 'Silverstone',
 'Black/Gray Poplar Wood Trim']

In [15]:
exterior = vehicle_info[1::6]
exterior = [el.replace('Exterior Color', '').strip() for el in exterior]
exterior

['Moonlight Blue Metallic',
 'Nano Gray Metallic',
 'Black',
 'White',
 'Black',
 'White',
 'Black',
 'Mythos Black Metallic',
 'Mythos Black Metallic',
 'Graphite Gray Metallic',
 'White',
 'Bluestone Metallic',
 'Melbourne Red Metallic',
 'Nterlagos Blue Metallic',
 'Dark Graphite Metallic']

In [16]:
drivetrain = vehicle_info[4::6]
drivetrain = [el.replace('Drivetrain', '').strip() for el in drivetrain]
drivetrain

['AWD',
 'AWD',
 'AWD',
 'AWD',
 'AWD',
 'AWD',
 'AWD',
 'AWD',
 'AWD',
 'AWD',
 'AWD',
 'AWD',
 'AWD',
 'RWD',
 'AWD']

In [17]:
# Dealership Info Dictionary
dealerships = {
    'Bostonyan Auto Group': {
        'url': 'https://www.bostonyanautogroup.com/view-inventory',
        'pagination_url': '',
        'dealership_name': 'Bostonyan Auto Group',
        'address': '119 Worcester St',
        'zipcode': '01760',
        'city': 'Natick',
        'state': 'MA'
    },
    'Direct Auto Mecca': {
        'url': 'https://www.directautomecca.com/view-inventory.aspx',
        'pagination_url': 'https://www.directautomecca.com/inventory.aspx?_new=true&_used=true&_page=2',
        'dealership_name': 'Direct Auto Mecca',
        'address': '154 Waverly Street',
        'zipcode': '01760',
        'city': 'Natick',
        'state': 'MA'
    },
    'Fafama Auto Sales': {
        'url': 'https://www.fafama.com/used-cars.aspx',
        'pagination_url': 'https://www.fafama.com/inventory.aspx?_used=true&_page=2',
        'dealership_name': 'Fafama Auto Sales',
        'address': '5 Cape Road',
        'zipcode': '01757',
        'city': 'Milford',
        'state': 'MA'
    },
    'Newton Automotive Sales': {
        'url': 'https://www.newtonautoandsales.com/cars-for-sale',
        'pagination_url': 'https://www.newtonautoandsales.com/cars-for-sale?PageNumber=2&Sort=MakeAsc&StockNumber=&Condition=&BodyStyle=&Make=&MaxPrice=&Mileage=&SoldStatus=AllVehicles&StockNumber=',
        'dealership_name': 'Newton Automotive Sales',
        'address': '249 Centre Street',
        'zipcode': '02458',
        'city': 'Newton',
        'state': 'MA'
    } 
}

In [18]:
c = parse_dealership.get_newton_auto_inventory_data(soup, dealerships['Newton Automotive Sales'])

In [19]:
c

,title,year,make,models,vehicle_type,vehicle_mileage,price,exterior_color,interior_color,transmission,engine,drivetrain,vin,dealership_name,dealership_address,dealership_zipcode,dealership_city,dealership_state,inventory_url,scraped_date
0,2018 Audi A4 2.0T quattro Premium Plus,2018,Audi,A4 2.0T quattro Premium Plus,None,15679,36590,Moonlight Blue Metallic,Nougat Brown Interior,Automatic 7-Speed,2.0L I4 Turbocharger,AWD,None,Newton Automotive Sales,249 Centre Street,02458,Newton,MA,https://www.newtonautoandsales.com/cars-for-sale,2022-03-13 19:34:31.631532
1,2017 Audi TT 2.0T quattro,2017,Audi,TT 2.0T quattro,None,29520,39480,Nano Gray Metallic,"Rock Gray, Fine Nappa Leather",Automatic 6-Speed,I4 2.0L Turbocharger,AWD,None,Newton Automotive Sales,249 Centre Street,02458,Newton,MA,https://www.newtonautoandsales.com/cars-for-sale,2022-03-13 19:34:31.631532
2,2018 Audi Q7 3.0T quattro Prestige,2018,Audi,Q7 3.0T quattro Prestige,None,53912,48560,Black,Brown,Automatic 8-Speed,V6 3.0L Supercharger,AWD,None,Newton Automotive Sales,249 Centre Street,02458,Newton,MA,https://www.newtonautoandsales.com/cars-for-sale,2022-03-13 19:34:31.631532
3,2017 Audi A4 2.0T quattro Premium Plus,2017,Audi,A4 2.0T quattro Premium Plus,None,53298,29350,White,Beige,Automatic 7-Speed,I4 2.0L Turbocharger,AWD,None,Newton Automotive Sales,249 Centre Street,02458,Newton,MA,https://www.newtonautoandsales.com/cars-for-sale,2022-03-13 19:34:31.631532
4,2018 Audi SQ5 3.0T quattro Prestige,2018,Audi,SQ5 3.0T quattro Prestige,None,99106,36850,Black,Black,Automatic 8-Speed,V6 3.0L Turbocharger,AWD,None,Newton Automotive Sales,249 Centre Street,02458,Newton,MA,https://www.newtonautoandsales.com/cars-for-sale,2022-03-13 19:34:31.631532
5,2013 Audi A4 2.0T quattro Premium,2013,Audi,A4 2.0T quattro Premium,None,54283,17980,White,Black,Automatic 8-Speed,I4 2.0L Turbocharger,AWD,None,Newton Automotive Sales,249 Centre Street,02458,Newton,MA,https://www.newtonautoandsales.com/cars-for-sale,2022-03-13 19:34:31.631532
6,2018 Audi S4 3.0T quattro Prestige,2018,Audi,S4 3.0T quattro Prestige,None,62800,41580,Black,Black,Automatic 8-Speed,V6 3.0L Turbocharger,AWD,None,Newton Automotive Sales,249 Centre Street,02458,Newton,MA,https://www.newtonautoandsales.com/cars-for-sale,2022-03-13 19:34:31.631532
7,2018 Audi S7 4.0T quattro Premium Plus,2018,Audi,S7 4.0T quattro Premium Plus,None,85521,48760,Mythos Black Metallic,Black,Automatic 7-Speed,V8 4.0L Twin Turbocharger,AWD,None,Newton Automotive Sales,249 Centre Street,02458,Newton,MA,https://www.newtonautoandsales.com/cars-for-sale,2022-03-13 19:34:31.631532
8,2020 Audi Q5 2.0T quattro Premium Plus,2020,Audi,Q5 2.0T quattro Premium Plus,None,29709,47480,Mythos Black Metallic,"Nougat Brown, Leather Seating",Automatic 7-Speed,I4 2.0L Turbocharger,AWD,None,Newton Automotive Sales,249 Centre Street,02458,Newton,MA,https://www.newtonautoandsales.com/cars-for-sale,2022-03-13 19:34:31.631532
9,2018 Audi Q7 3.0T quattro Premium Plus,2018,Audi,Q7 3.0T quattro Premium Plus,None,99848,34980,Graphite Gray Metallic,Black,Automatic 8-Speed,V6 3.0L Supercharger,AWD,None,Newton Automotive Sales,249 Centre Street,02458,Newton,MA,https://www.newtonautoandsales.com/cars-for-sale,2022-03-13 19:34:31.631532


In [93]:
def days_since_last_scrape(dealership_name, db_name, tbl_name):
    conn = sqlite3.connect(db_name)
    sql_query = f"SELECT MIN(scraped_date) AS min_scraped_date from {tbl_name} WHERE dealership_name = '{dealership_name}'"
    result = pd.read_sql_query(sql_query, conn)
    past = pd.to_datetime(result['min_scraped_date'])[0]
    now = datetime.now()
    duration = now - past
    duration_in_s = duration.total_seconds()
    days  = divmod(duration_in_s, 86400)[0]
    
    return days

In [72]:
x = last_scraped_date('Direct Auto Mecca','cars.db', 'inventory_staging')

In [73]:
x

3.0

In [74]:
y = last_scraped_date('Blah','cars.db', 'inventory_staging')

In [75]:
y

nan

In [78]:
import numpy as np

In [81]:
np.isnan(y)

True

In [82]:
np.isnan(x)

False

In [80]:
type(y)

float

In [83]:
type(x)

float

In [84]:
if y > 14:
    print('hello')

In [98]:
# Check data
conn = sqlite3.connect('cars.db')
sql_query = "SELECT dealership_name, count(*) AS num_of_vehicles FROM inventory_staging GROUP BY 1"
result = pd.read_sql_query(sql_query, conn)
result

DatabaseError: Execution failed on sql 'SELECT dealership_name, count(*) AS num_of_vehicles FROM inventory_staging GROUP BY 1': no such table: inventory_staging

,dealership_name,num_of_vehicles
0,Bostonyan Auto Group,18
1,Direct Auto Mecca,149
2,Fafama Auto Sales,145
3,Newton Automotive Sales,15


In [89]:
pagination_url = dealerships['Newton Automotive Sales']['pagination_url']
page_counter = 3
pagination_url = re.sub('PageNumber=[0-9+]', f'PageNumber={page_counter}', pagination_url)   
pagination_url

'https://www.newtonautoandsales.com/cars-for-sale?PageNumber=3&Sort=MakeAsc&StockNumber=&Condition=&BodyStyle=&Make=&MaxPrice=&Mileage=&SoldStatus=AllVehicles&StockNumber='

In [90]:
response = requests.get(pagination_url, headers = headers)
soup_pagination = BeautifulSoup(response.text, "html.parser")   
pi.parseColumn(soup_pagination, 'h3', 'vehicle-snapshot__title')

['2010 Mercedes-Benz GLK GLK 350 4MATIC',
 '2015 Mercedes-Benz E-Class E 63 AMG S-Model',
 '2016 Porsche Macan S',
 '2013 Porsche Cayenne GTS',
 '2016 Porsche Cayenne S',
 '2021 Tesla Model 3 Long Range']

In [91]:
pagination_url = dealerships['Newton Automotive Sales']['pagination_url']
page_counter = 4
pagination_url = re.sub('PageNumber=[0-9+]', f'PageNumber={page_counter}', pagination_url)   
pagination_url

'https://www.newtonautoandsales.com/cars-for-sale?PageNumber=4&Sort=MakeAsc&StockNumber=&Condition=&BodyStyle=&Make=&MaxPrice=&Mileage=&SoldStatus=AllVehicles&StockNumber='

In [92]:
response = requests.get(pagination_url, headers = headers)
soup_pagination = BeautifulSoup(response.text, "html.parser")   
pi.parseColumn(soup_pagination, 'h3', 'vehicle-snapshot__title')

[]

In [21]:
TABLE_NAME = 'inventory_staging'
DB_NAME = 'cars.db'

#Connecting to sqlite
conn = sqlite3.connect(DB_NAME)

#Creating a cursor object using the cursor() method
cursor = conn.cursor()

#Doping EMPLOYEE table if already exists
cursor.execute(f"DROP TABLE {TABLE_NAME}")
print("Table dropped... ")

#Commit your changes in the database
conn.commit()

#Closing the connection
conn.close()

Table dropped... 


In [28]:
# Check data
conn = sqlite3.connect('cars.db')
sql_query = "SELECT dealership_name, count(*) AS num_of_vehicles FROM inventory_staging GROUP BY 1"
result = pd.read_sql_query(sql_query, conn)
result

,dealership_name,num_of_vehicles
0,Bostonyan Auto Group,18
1,Direct Auto Mecca,146
2,Fafama Auto Sales,132
3,Newton Automotive Sales,35


In [50]:
# Check data
conn = sqlite3.connect('cars.db')
sql_query = "SELECT dealership_name, count(*) AS num_of_vehicles FROM inventory_staging GROUP BY 1"
result = pd.read_sql_query(sql_query, conn)
result

,dealership_name,num_of_vehicles
0,Blasius Boston,289
1,Bostonyan Auto Group,18
2,Direct Auto Mecca,146
3,Fafama Auto Sales,132
4,Newton Automotive Sales,35


In [51]:
conn = sqlite3.connect('cars.db')
sql_query = "SELECT * FROM inventory_staging WHERE dealership_name = 'Blasius Boston'"
result = pd.read_sql_query(sql_query, conn)
result.isnull().sum()

title                   0
year                    0
make                    0
models                  0
vehicle_type          288
vehicle_mileage         0
price                   1
exterior_color          0
interior_color          5
transmission            2
engine                  0
drivetrain              0
vin                     0
dealership_name         0
dealership_address      0
dealership_zipcode      0
dealership_city         0
dealership_state        0
inventory_url           0
scraped_date            0
dtype: int64

In [52]:
pd.set_option('display.max_rows', None)

In [53]:
result.to_csv('cars.csv')

In [106]:
def days_since_last_scrape(dealership_name, db_name, tbl_name):
    # First check if the table exists
    conn = sqlite3.connect('cars.db')
    check_table = f"SELECT name FROM sqlite_master WHERE type='table' AND name='{tbl_name}';"
    tbl = pd.read_sql_query(check_table, conn)
    
    if (tbl['name'].empty):
        return np.nan
    else:
        conn = sqlite3.connect(db_name)
        sql_query = f"SELECT MIN(scraped_date) AS min_scraped_date from {tbl_name} WHERE dealership_name = '{dealership_name}'"
        result = pd.read_sql_query(sql_query, conn)
        past = pd.to_datetime(result['min_scraped_date'])[0]
        now = datetime.datetime.now()
        duration = now - past
        duration_in_s = duration.total_seconds()
        days  = divmod(duration_in_s, 86400)[0]
        
        return days

In [107]:
y = days_since_last_scrape('Fafama Auto Sales','cars.db', 'inventory_staging')
y

nan

In [48]:
TABLE_NAME = 'inventory_staging'
DB_NAME = 'cars.db'

#Connecting to sqlite
conn = sqlite3.connect(DB_NAME)

#Creating a cursor object using the cursor() method
cursor = conn.cursor()

#Doping EMPLOYEE table if already exists
cursor.execute(f"DELETE FROM {TABLE_NAME} WHERE dealership_name = 'Blasius Boston'")

#Commit your changes in the database
conn.commit()

#Closing the connection
conn.close()

In [49]:
# Check data
conn = sqlite3.connect('cars.db')
sql_query = "SELECT dealership_name, count(*) AS num_of_vehicles FROM inventory_staging GROUP BY 1"
result = pd.read_sql_query(sql_query, conn)
result

,dealership_name,num_of_vehicles
0,Bostonyan Auto Group,18
1,Direct Auto Mecca,146
2,Fafama Auto Sales,132
3,Newton Automotive Sales,35


In [30]:
print(1, 2, 3, 4)

1 2 3 4


In [36]:
pagination_url = 'https://www.blasiusboston.com/used-cars-holliston-ma?page=10'

In [37]:
page_counter = 11
pagination_url = re.sub('page=[0-9]+', f'page={page_counter}', pagination_url)   

In [38]:
pagination_url

'https://www.blasiusboston.com/used-cars-holliston-ma?page=11'

In [54]:
conn = sqlite3.connect('cars.db')
sql_query = "SELECT * FROM inventory_staging WHERE dealership_name = 'Fafama Auto Sales'"
result = pd.read_sql_query(sql_query, conn)
result.to_csv('cars.csv')

In [55]:
result.isnull().sum()

title                   0
year                    0
make                    0
models                  0
vehicle_type            0
vehicle_mileage         0
price                   9
exterior_color          0
interior_color          0
transmission            0
engine                  1
drivetrain            132
vin                     0
dealership_name         0
dealership_address      0
dealership_zipcode      0
dealership_city         0
dealership_state        0
inventory_url           0
scraped_date            0
dtype: int64

In [56]:
conn = sqlite3.connect('cars.db')
sql_query = "SELECT * FROM inventory_staging WHERE dealership_name = 'Direct Auto Mecca'"
result = pd.read_sql_query(sql_query, conn)
result.to_csv('cars.csv')

In [57]:
result.isnull().sum()

title                   0
year                    0
make                    3
models                  0
vehicle_type          145
vehicle_mileage         0
price                   7
exterior_color          0
interior_color          0
transmission            0
engine                  0
drivetrain            146
vin                     0
dealership_name         0
dealership_address      0
dealership_zipcode      0
dealership_city         0
dealership_state        0
inventory_url           0
scraped_date            0
dtype: int64

In [58]:
TABLE_NAME = 'inventory_staging'
DB_NAME = 'cars.db'

#Connecting to sqlite
conn = sqlite3.connect(DB_NAME)

#Creating a cursor object using the cursor() method
cursor = conn.cursor()

#Doping EMPLOYEE table if already exists
cursor.execute(f"DELETE FROM {TABLE_NAME} WHERE dealership_name = 'Direct Auto Mecca'")

#Commit your changes in the database
conn.commit()

#Closing the connection
conn.close()
